In [1]:
import numpy as np
import pandas as pd
import os

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

data_path = '/content/drive/My Drive/databank/'

Mounted at /content/drive/


In [3]:
df_review = pd.read_csv(data_path + 'clean_review_sg.csv')

df_review.head()

,listing_id,id,reviewer_id,comments
0,275343.00,4223592.00,5492044,A newly renovated and clean apartment. Conveni...
1,275343.00,8818243.00,7938877,I am so glad that i chose the place. Joyce and...
2,275343.00,26820048.00,18071761,I enjoyed the time shared with the Woons. Joyc...
3,275343.00,28258248.00,26579851,"The Woons are nice welcoming people, I had a g..."
4,275343.00,34945181.00,29892169,I stayed with the Woon family for 3 weeks in M...


In [4]:
%%capture
!pip install transformers
from transformers import pipeline

# sentiment analysis task with twitter roberta model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

In [18]:
# Define a function to handle errors
def safe_sentiment(text):
    try:
        return sentiment_pipeline(text, truncation=True, max_length=514)
    except Exception as e:
        return [{"label": "error", "score": 0.0, "error_message": str(e)}]

In [23]:
%%time
# Predict sentiment
transformer_results = df_review['comments'].apply(safe_sentiment)

sentiment_label = transformer_results.apply(lambda x: x[0]['label'])

# Create sentiment category
sent_value = ['Negative', 'Neutral', 'Positive', None]
sent_condition = [sentiment_label == 'LABEL_0',
                  sentiment_label == 'LABEL_1',
                  sentiment_label == 'LABEL_2',
                  sentiment_label == 'error'
                  ]

sentiment_label = np.select(sent_condition, sent_value)

df_review['sentiment'] = sentiment_label

df_review.to_csv(data_path + 'clean_review_sg_with_sentiment.csv', index = False)

CPU times: user 2h 33min 12s, sys: 6.82 s, total: 2h 33min 19s
Wall time: 2h 34min 21s
